In [ ]:
import beamtest_analysis_helper as helper
from glob import glob
from natsort import natsorted
from tqdm.notebook import tqdm
import hist
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors
import mplhep as hep
hep.style.use('CMS')
import random
import importlib
importlib.reload(helper)

In [ ]:
columns_to_read = ['evt', 'board', 'row', 'col', 'toa', 'tot', 'cal']
names = ["ET2_EPIR_Pair1", "ET2p01_BAR_4", "ET2p01_BAR_5", "ET2_EPIR_Pair4"]
original_files = glob('/home/daq/ETROC2/ETROC-Data/DESYFeb2024_Run_34_feather/*feather')
files = random.sample(original_files, k=20)
# files = original_files
dut1_id = 1
dut2_id = 3
ref_id = 2

last_evt = 0
dataframes = []
for idx, ifile in enumerate(files):
    tmp_df = pd.read_feather(ifile, columns=columns_to_read)
    if tmp_df.empty:
        continue
    if idx > 0:
        tmp_df['evt'] += last_evt
    last_evt += np.uint64(tmp_df['evt'].unique()[-1])
    ## Selecting good hits
    tdc_cuts = {}
    for idx in [0,1,2,3]:
        # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
        if idx == 0:
            tdc_cuts[idx] = [tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]-50, tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]+50,  400, 500, 100, 250]
        elif idx == ref_id:
            tdc_cuts[idx] = [tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]-50, tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]+50,  0, 1100, 80, 200]
        else:
            tdc_cuts[idx] = [tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]-50, tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]+50,  0, 1100, 0, 600]
    filtered_df = helper.tdc_event_selection(tmp_df, tdc_cuts_dict=tdc_cuts,select_by_hit=True)
    del tmp_df
    if filtered_df.empty:
        continue
    event_board_counts = filtered_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
    event_selection_col = None

    trig_selection = (event_board_counts[0] == 1)
    ref_selection = (event_board_counts[ref_id] == 1)
    event_selection_col = trig_selection & ref_selection

    selected_event_numbers = event_board_counts[event_selection_col].index
    selected_subset_df = filtered_df[filtered_df['evt'].isin(selected_event_numbers)]
    selected_subset_df.reset_index(inplace=True, drop=True)

    dataframes.append(selected_subset_df)
    del event_board_counts, selected_event_numbers, selected_subset_df, event_selection_col, filtered_df

df = pd.concat(dataframes)
df.reset_index(inplace=True, drop=True)
del dataframes



In [ ]:
pixel_dict = {
    0: (7,4),
}
for board, pix in pixel_dict.items():
    filtered_df = df.loc[~( (df['board'] == board) & ((df['row'] != pix[0]) | (df['col'] != pix[1])) )].copy()
del df

In [ ]:
helper.plot_occupany_map(df, chipLabels=[0,1,2,3], chipNames=names, fig_title=names, fig_tag='')
h_inclusive = helper.return_hist(df, names, [0,1,2,3], hist_bins=[100, 128, 128])
for iboard in [0,1,2,3]:
    helper.plot_1d_TDC_histograms(h_inclusive, names[iboard], names[iboard], names[iboard], './', save=False,
                                tag="", fig_tag="", slide_friendly=True)
del h_inclusive

In [ ]:
#### Efficiency plot
# Case 1: a single hit on trig and ref, then TOA correlation between trig and ref
# Case 2: a single hit on trig and ref, then multiple hits on DUT1 => TOA correction trig and ref, trig - DUT1, ref - DUT1
# Case 3: a single hit on trig and ref and DUT2, then TOA correlation between trig and ref
# Case 4: a single hit on trig and ref and DUT2, then mutiple hits on DUT1 => TOA correction trig and ref, trig - DUT1, ref - DUT1


####
# To make occupancy plot - all inclusive
# Case 1: No hits on DUT2
# Case 2: 1 hit on DUT2
# Case 3: >= 2 hit on DUT2

# Case 1

In [ ]:
filtered_df = df

In [ ]:
# Case 1: a single hit on trig and ref, then TOA correlation between trig and ref

event_board_counts = filtered_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
event_selection_col = None

trig_selection = (event_board_counts[0] == 1)
ref_selection = (event_board_counts[ref_id] == 1)
event_selection_col = trig_selection & ref_selection

case1_df = filtered_df[filtered_df['evt'].isin(event_board_counts[event_selection_col].index)]
case1_df.reset_index(inplace=True, drop=True)

In [ ]:
diff_row = abs(case1_df[case1_df['board'] == 0]['row'].values - case1_df[case1_df['board'] == ref_id]['row'].values)
diff_col = abs(case1_df[case1_df['board'] == 0]['col'].values - case1_df[case1_df['board'] == ref_id]['col'].values)

In [ ]:
## Select event numbers where trig - ref colinearity has been satisfied
trig_ref_colinear_evts = case1_df['evt'].unique()[(diff_row <= 1) & (diff_col <= 1)]

In [ ]:
print(trig_ref_colinear_evts.size)

### TDC Correlation

In [ ]:
# are we plotting correlation from same event? This looks like raw scatter plot

In [ ]:
plot_case1_df = case1_df.loc[(case1_df['board'] == 0) | (case1_df['board'] == ref_id)]
helper.plot_TDC_correlation_scatter_matrix(input_df=plot_case1_df, chip_names=names, single_hit=False, colinear=True, colinear_cut=1, save=False)
del plot_case1_df

# Case 2

In [ ]:
# Case 2: a single hit on trig and ref, then multiple hits on DUT1 => TOA correlation trig and ref, trig - DUT1, ref - DUT1, and efficiency of only events with colinear hits
dut1_df = helper.return_broadcast_dataframe(input_df=filtered_df, trig_board_id=0, ref_board_id=ref_id, dut_board_id=dut1_id, trig_dut=True, ref_dut=False)

In [ ]:
colinear_dut1_df = dut1_df[dut1_df['evt'].isin(trig_ref_colinear_evts)].reset_index(drop=True)

In [ ]:
### Board-level Efficiency of DUT w/o considering colinear
100*(dut1_df['evt'].nunique())/case1_df['evt'].nunique()

In [ ]:
### Board-level Efficiency of DUT w/ considering colinear of trigger and reference boards
# 100*colinear_dut1_df['evt'].nunique()/case1_df['evt'].nunique()
100*colinear_dut1_df['evt'].nunique()/trig_ref_colinear_evts.size

In [ ]:
diff_row = colinear_dut1_df[colinear_dut1_df['board'] == 0]['row'].values - colinear_dut1_df[colinear_dut1_df['board'] == dut1_id]['row'].values
diff_col = colinear_dut1_df[colinear_dut1_df['board'] == 0]['col'].values - colinear_dut1_df[colinear_dut1_df['board'] == dut1_id]['col'].values

In [ ]:
## Select event numbers where trig - dut colinearity has been satisfied
trig_dut_colinear_df = colinear_dut1_df[colinear_dut1_df['board'] == 0][(diff_row <= 1) & (diff_col <= 1)]
trig_dut_colinear_evts = trig_dut_colinear_df['evt'].unique()
print(trig_dut_colinear_evts.size,colinear_dut1_df['evt'].nunique(),trig_ref_colinear_evts.size,dut1_df['evt'].nunique(),case1_df['evt'].nunique())

In [ ]:
# 100*trig_dut_colinear_df['evt'].nunique()/case1_df['evt'].nunique()
100*trig_dut_colinear_df['evt'].nunique()/trig_ref_colinear_evts.size

In [ ]:
colinear_case1_df = case1_df[case1_df['evt'].isin(trig_ref_colinear_evts)].reset_index(drop=True)
hits_count_by_col_row_board = colinear_case1_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()
# hits_count_by_col_row_board = colinear_dut1_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()

hits_count_by_col_row_board = hits_count_by_col_row_board.rename(columns={'evt': 'hits'})
denominator = hits_count_by_col_row_board[hits_count_by_col_row_board['board'] == 0].pivot_table(
    index='row',
    columns='col',
    values='hits',
    fill_value=0  # Fill missing values with 0 (if any)
)

In [ ]:
# hits_count_by_col_row_board = trig_dut_colinear_df.groupby(['col', 'row'])['evt'].count().reset_index()
colinear_case2_df = case1_df[case1_df['evt'].isin(trig_dut_colinear_evts)].reset_index(drop=True)
hits_count_by_col_row_board = colinear_case2_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()
hits_count_by_col_row_board = hits_count_by_col_row_board.rename(columns={'evt': 'hits'})
numerator = hits_count_by_col_row_board[hits_count_by_col_row_board['board'] == 0].pivot_table(
    index='row',
    columns='col',
    values='hits',
    fill_value=0  # Fill missing values with 0 (if any)
)

In [ ]:
eff_table = 100*numerator/denominator
eff_table = eff_table.fillna(0)

# Create a heatmap to visualize the count of hits
fig, ax = plt.subplots(dpi=100, figsize=(20, 20))
im = ax.imshow(eff_table, cmap="viridis", interpolation="nearest")

# Add color bar
cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Efficiency', fontsize=20)
cbar.ax.tick_params(labelsize=20)

for i in range(16):
    for j in range(16):
        try:
            value = eff_table.iloc[i, j]
        except:
            value = -1
        if value == -1: continue
        text_color = 'black' if value > 0.5*(eff_table.values.max() + eff_table.values.min()) else 'white'
        text = str("{:.1f}%".format(value))
        plt.text(j, i, text, va='center', ha='center', color=text_color, fontsize=17)

hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_xlabel('Column (col)', fontsize=20)
ax.set_ylabel('Row (row)', fontsize=20)
ticks = range(0, 16)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_title(f"Efficiency of DUT projected to the trigger board based on 3-board analysis", loc="right", size=20)
ax.tick_params(axis='x', which='both', length=5, labelsize=17)
ax.tick_params(axis='y', which='both', length=5, labelsize=17)
ax.invert_xaxis()
ax.invert_yaxis()
plt.minorticks_off()

# fig.savefig('dut1_3board_eff_map.png')
# fig.savefig('dut1_3board_eff_map.pdf')

### TDC correlation

In [ ]:
helper.plot_TDC_correlation_scatter_matrix(input_df=dut1_df, chip_names=names, single_hit=False, colinear=True, save=False)

# Case 3

In [ ]:
# Case 3: a single hit on trig and ref and DUT2, then TOA correlation between trig and ref

event_board_counts = filtered_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
event_selection_col = None

trig_selection = (event_board_counts[0] == 1)
ref_selection = (event_board_counts[ref_id] == 1)
ref_2nd_selection = (event_board_counts[dut2_id] == 1)
event_selection_col = trig_selection & ref_selection & ref_2nd_selection

case3_df = filtered_df[filtered_df['evt'].isin(event_board_counts[event_selection_col].index)]
case3_df.reset_index(inplace=True, drop=True)

In [ ]:
diff_row1 = abs(case3_df[case3_df['board'] == 0]['row'].values - case3_df[case3_df['board'] == ref_id]['row'].values)
diff_col1 = abs(case3_df[case3_df['board'] == 0]['col'].values - case3_df[case3_df['board'] == ref_id]['col'].values)

diff_row2 = abs(case3_df[case3_df['board'] == 0]['row'].values - case3_df[case3_df['board'] == dut2_id]['row'].values)
diff_col2 = abs(case3_df[case3_df['board'] == 0]['col'].values - case3_df[case3_df['board'] == dut2_id]['col'].values)

In [ ]:
## Select event numbers where trig - ref colinearity has been satisfied
trig_ref_notInterestDut_colinear_evts = case3_df['evt'].unique()[(diff_row1 <= 1) & (diff_col1 <= 1) & (diff_row2 <= 1) & (diff_col2 <= 1)]

### TDC Correlation

In [ ]:
plot_case3_df = case3_df.loc[(case3_df['board'] == 0) | (case3_df['board'] == ref_id)]
helper.plot_TDC_correlation_scatter_matrix(input_df=plot_case3_df, chip_names=names, single_hit=False, colinear=False, save=False)
del plot_case3_df

# Case 4

In [ ]:
# Case 4: a single hit on trig and ref and DUT2, then mutiple hits on DUT1 => TOA correction trig and ref, trig - DUT1, ref - DUT1
dut1_df = helper.return_broadcast_dataframe(input_df=filtered_df, trig_board_id=0, ref_board_id=ref_id, dut_board_id=dut1_id, second_ref_board_id=dut2_id, trig_dut=True, ref_dut=False)

In [ ]:
colinear_dut1_df = dut1_df[dut1_df['evt'].isin(trig_ref_notInterestDut_colinear_evts)].reset_index(drop=True)

In [ ]:
### Board-level Efficiency of DUT w/o considering colinear
100*(dut1_df['evt'].nunique())/case3_df['evt'].nunique()

In [ ]:
### Board-level Efficiency of DUT w/ considering colinear of trigger and reference boards
# 100*colinear_dut1_df['evt'].nunique()/case3_df['evt'].nunique()
100*colinear_dut1_df['evt'].nunique()/trig_ref_notInterestDut_colinear_evts.size

In [ ]:
diff_row = colinear_dut1_df[colinear_dut1_df['board'] == 0]['row'].values - colinear_dut1_df[colinear_dut1_df['board'] == dut1_id]['row'].values
diff_col = colinear_dut1_df[colinear_dut1_df['board'] == 0]['col'].values - colinear_dut1_df[colinear_dut1_df['board'] == dut1_id]['col'].values

In [ ]:
## Select event numbers where trig - dut colinearity has been satisfied
trig_dut_colinear_df = colinear_dut1_df[colinear_dut1_df['board'] == 0][(diff_row <= 1) & (diff_col <= 1)]
trig_dut_colinear_evts = trig_dut_colinear_df['evt'].unique()

In [ ]:
# 100*trig_dut_colinear_df['evt'].nunique()/case3_df['evt'].nunique()
100*trig_dut_colinear_df['evt'].nunique()/trig_ref_notInterestDut_colinear_evts.size

In [ ]:
# # hits_count_by_col_row_board = case3_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()
# hits_count_by_col_row_board = colinear_dut1_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()
colinear_case3_df = case3_df[case3_df['evt'].isin(trig_ref_notInterestDut_colinear_evts)].reset_index(drop=True)
hits_count_by_col_row_board = colinear_case3_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()

hits_count_by_col_row_board = hits_count_by_col_row_board.rename(columns={'evt': 'hits'})
denominator = hits_count_by_col_row_board[hits_count_by_col_row_board['board'] == 0].pivot_table(
    index='row',
    columns='col',
    values='hits',
    fill_value=0  # Fill missing values with 0 (if any)
)

In [ ]:
# hits_count_by_col_row_board = trig_dut_colinear_df.groupby(['col', 'row'])['evt'].count().reset_index()
# hits_count_by_col_row_board = hits_count_by_col_row_board.rename(columns={'evt': 'hits'})
# numerator = hits_count_by_col_row_board.pivot_table(
#     index='row',
#     columns='col',
#     values='hits',
#     fill_value=0  # Fill missing values with 0 (if any)
# )

colinear_case4_df = case3_df[case3_df['evt'].isin(trig_dut_colinear_evts)].reset_index(drop=True)
hits_count_by_col_row_board = colinear_case4_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()
hits_count_by_col_row_board = hits_count_by_col_row_board.rename(columns={'evt': 'hits'})
numerator = hits_count_by_col_row_board[hits_count_by_col_row_board['board'] == 0].pivot_table(
    index='row',
    columns='col',
    values='hits',
    fill_value=0  # Fill missing values with 0 (if any)
)

In [ ]:
eff_table = 100*numerator/denominator
eff_table = eff_table.fillna(0)

# Create a heatmap to visualize the count of hits
fig, ax = plt.subplots(dpi=100, figsize=(20, 20))
im = ax.imshow(eff_table, cmap="viridis", interpolation="nearest")

# Add color bar
cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Efficiency', fontsize=20)
cbar.ax.tick_params(labelsize=20)

for i in range(16):
    for j in range(16):
        value = eff_table.iloc[i, j]
        if value == -1: continue
        text_color = 'black' if value > 0.5*(eff_table.values.max() + eff_table.values.min()) else 'white'
        text = str("{:.1f}%".format(value))
        plt.text(j, i, text, va='center', ha='center', color=text_color, fontsize=17)

hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_xlabel('Column (col)', fontsize=20)
ax.set_ylabel('Row (row)', fontsize=20)
ticks = range(0, 16)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_title(f"Efficiency of DUT projected to the trigger board based on 4-board analysis", loc="right", size=20)
ax.tick_params(axis='x', which='both', length=5, labelsize=17)
ax.tick_params(axis='y', which='both', length=5, labelsize=17)
ax.invert_xaxis()
ax.invert_yaxis()
plt.minorticks_off()

# fig.savefig('dut1_3board_eff_map.png')

### TDC correlation

In [ ]:
helper.plot_TDC_correlation_scatter_matrix(input_df=dut1_df, chip_names=names, single_hit=False, colinear=True, save=False)